# Подготовка данных для нейронной сети

Используется соотношение открытого интереса по Call и Put опционам.  
День недели.  
Label в виде 'up', 'down'.  
Запись результатов в файл `nn_features_and_target.csv`  

In [1]:
from pathlib import Path
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import sys
import importlib

import candle_code_Lihovidov

sys.dont_write_bytecode = True

In [2]:
tiker: str = 'RTS'
db_path: Path = Path(fr'c:\Users\Alkor\gd\data_quote_db\{tiker}_futures_options_day_pj1.db')

# Подключение к базе данных
conn = sqlite3.connect(db_path)

# Чтение данных из БД в DataFrame
df_f = pd.read_sql_query("SELECT `TRADEDATE`, `OPEN`, `LOW`, `HIGH`, `CLOSE`, `OPENPOSITION`, `SHORTNAME`, `LSTTRADE` FROM Futures", conn)
df_o = pd.read_sql_query("SELECT `TRADEDATE`, `OPENPOSITION`, `NAME`, `LSTTRADE`, `OPTIONTYPE`, `STRIKE` FROM Options", conn)

# Закрытие соединения
conn.close()

df_f[['TRADEDATE', 'LSTTRADE']] = df_f[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)
df_o[['TRADEDATE', 'LSTTRADE']] = df_o[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)

df_o = df_o[df_o.TRADEDATE < df_o.LSTTRADE]
# df_f['up_down'] = df_f[['OPEN', 'CLOSE']].apply(lambda x: 1 if (x.CLOSE > x.OPEN) else 0, axis=1)  # Добавление колонки направления свечи
df_f['up_down'] = df_f[['OPEN', 'CLOSE']].apply(lambda x: 'up' if (x.CLOSE > x.OPEN) else 'down', axis=1)  # Добавление колонки направления свечи
df_f['next_ud'] = df_f.up_down.shift(-1)  # Добавление колонки с направление следующей свечи
df_f['next_name'] = df_f.SHORTNAME.shift(-1)  # Добавление колонки с именем следующей свечи

# Фильтрация строк, где значения в столбцах SHORTNAME и next_name равны (исключение переходов контракта)
# Реализовано на более позднем этапе
# df_f = df_f[df_f['SHORTNAME'] == df_f['next_name']]

# print(df_f.to_string(max_rows=4, max_cols=10))
# df_f = df_f.dropna()
# df_f[['next_ud']] = df_f[['next_ud']].astype(int)

# Сортировка по полю даты и сброс индекса
df_f = df_f.sort_values(by='TRADEDATE').reset_index(drop=True)
df_o = df_o.sort_values(by='TRADEDATE').reset_index(drop=True)

# Вывод DataFrame
# print(df_f)
# print(df_o)
print(df_f.to_string(max_rows=4, max_cols=20))
print(df_f.shape)
print('\n', df_o.to_string(max_rows=4, max_cols=10))
print(df_o.shape)

      TRADEDATE      OPEN       LOW      HIGH     CLOSE  OPENPOSITION SHORTNAME   LSTTRADE up_down next_ud next_name
0    2015-01-05   76930.0   72470.0   78980.0   74600.0        751996  RTS-3.15 2015-03-16    down    down  RTS-3.15
1    2015-01-06   74470.0   71200.0   74610.0   73480.0        731236  RTS-3.15 2015-03-16    down      up  RTS-3.15
...         ...       ...       ...       ...       ...           ...       ...        ...     ...     ...       ...
2403 2024-08-05  108710.0  105730.0  108850.0  106240.0         63506  RTS-9.24 2024-09-19    down    down  RTS-9.24
2404 2024-08-06  106170.0  105300.0  107990.0  106170.0         60676  RTS-9.24 2024-09-19    down    None      None
(2405, 11)

         TRADEDATE  OPENPOSITION      NAME   LSTTRADE OPTIONTYPE  STRIKE
0      2015-01-05         18004  RTS-3.15 2015-01-15          C  100000
1      2015-01-05             0  RTS-3.15 2015-03-16          C   45000
...           ...           ...       ...        ...        ...     .

In [3]:
step_strike = 2500

df_rez = pd.DataFrame()

for row in df_f.itertuples():  # Перебирает построчно DF с фьючерсами
    # Выбор из DF с опционами только опционов на дату фьючерса
    df = df_o[df_o.TRADEDATE == row.TRADEDATE]
    # print(df.to_string(max_rows=4, max_cols=10))
    
    # DF с опционами call
    df_p = (
        df.query('OPTIONTYPE ==  "P"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_p'})
    )
    # print(df_p.to_string(max_rows=4, max_cols=10))
    # print(df_p.shape)
    
    # DF с опционами put
    df_c = (
        df.query('OPTIONTYPE ==  "C"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_c'})
    )
    # print(df_c.to_string(max_rows=4, max_cols=10))
    # print(df_c.shape)
    
    # Временный DF со страйками (для последующего слияния), чтобы исключить пропуски страйков
    df_tmp = pd.DataFrame(columns=['STRIKE'])
    for st in range(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
        # Новое значение для добавления
        new_row = pd.DataFrame({'STRIKE': [st]})
        # Добавление новой строки в DataFrame с помощью concat
        df_tmp = pd.concat([df_tmp, new_row], ignore_index=True)
    # print(df_tmp.to_string(max_rows=4, max_cols=10))
    
    # Выполнение полного внешнего соединения DF'ов с опционами call и put по столбцу STRIKE
    merged_df = pd.merge(df_p, df_c, on='STRIKE', how='outer')
    merged_df = pd.merge(merged_df, df_tmp, on='STRIKE', how='outer')
    # Приведение типов с помощью infer_objects
    merged_df = merged_df.infer_objects(copy=False)
    
    merged_df = merged_df.fillna(0)  # Пустые значения заполняем "0"
    # Приведение типов к int
    merged_df[['STRIKE', 'oi_c', 'oi_p']] = merged_df[['STRIKE', 'oi_c', 'oi_p']].astype(int)
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    # print(merged_df.shape)
    
    # Переделываем колонку с ОИ по call в колонку с накопленной суммой ОИ
    merged_df['oi_c'] = merged_df['oi_c'].cumsum()
    # Переделываем колонку с ОИ по put в колонку с накопленной суммой ОИ в обратном порядке
    merged_df['oi_p'] = merged_df.iloc[::-1]['oi_p'].cumsum()[::-1]
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    
    # Записываем в переменную значение цены CLOSE фьючерса
    price_close = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'CLOSE'].values[0]
    # Расчитываем ближайший страйк к цене CLOSE
    nearest_strike = round(price_close / step_strike) * step_strike
    # print(price_close, nearest_strike)
    
    # Список индексов со значением nearest_strike из поля merged_df['STRIKE']
    index_lst = merged_df.index[merged_df['STRIKE'] == nearest_strike].tolist()
    # Известный индекс строки (берем 0)
    index_nearest = index_lst[0]
    # Определение диапазона строк
    start_index = max(0, index_nearest - 10)
    end_index = min(len(df), index_nearest + 10 + 1)
    # Получение 10 строк до и 10 строк после строки с известным индексом ближайшего страйка
    subset_df = merged_df.iloc[start_index:end_index]
    subset_df = subset_df.copy()
    # Создание колонки соотношений (разницы) отрытого интереса Call и Put
    subset_df['oi'] = subset_df.apply(lambda x: x.oi_p - x.oi_c if x.STRIKE < price_close else x.oi_c - x.oi_p, axis=1)
    # print(subset_df.to_string(max_rows=40, max_cols=10))
    # Извлекаем колонку 'oi' из subset_df
    column_oi_arr = subset_df['oi'].values
    # Нормализация значений в диапазон [0, 1]
    arr_min = np.min(column_oi_arr)
    arr_max = np.max(column_oi_arr)
    normalized_arr = (column_oi_arr - arr_min) / (arr_max - arr_min)
    
    normalized_arr = np.append(row.next_name, normalized_arr)
    normalized_arr = np.append(row.SHORTNAME, normalized_arr)
    normalized_arr = np.append(row.TRADEDATE, normalized_arr)
    
    # Добавляем день недели
    # print(row.TRADEDATE.weekday())
    if row.TRADEDATE.weekday() == 0:
        normalized_arr = np.append(normalized_arr, np.array([1, 0, 0, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 1:
        normalized_arr = np.append(normalized_arr, np.array([0, 1, 0, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 2:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 1, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 3:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 1, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 4:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 1, 0, 0]))
    elif row.TRADEDATE.weekday() == 5:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 0, 1, 0]))
    elif row.TRADEDATE.weekday() == 6:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 0, 0, 1]))
    # Добавляем target (label), следующий день повышение или падение
    column_as_row = np.append(normalized_arr, row.next_ud)
    # print(column_as_row)
    # print(type(column_as_row))
    # break
    # Добавляем колонку как строку в DataFrame df_rez
    # Для этого нам нужно транспонировать DataFrame, добавить строку и снова транспонировать обратно
    df_rez = df_rez.T
    df_rez[len(df_rez.columns)] = column_as_row
    df_rez = df_rez.T
    # print(df_rez)

    # break
# Преобразование колонки в тип datetime
df_rez[0] = pd.to_datetime(df_rez[0])
df_rez[0] = df_rez[0].dt.date  # Оставляем только дату
df_rez = df_rez.rename(columns={0: 'TRADEDATE', 1:'SHORTNAME', 2:'next_name', 31:'next_up/down'})

print(df_rez.to_string(max_rows=8, max_cols=30))
print(df_rez.shape)

       TRADEDATE SHORTNAME next_name                   3                   4                   5                   6                   7                   8                   9                   10                   11                   12                   13                   14  ...                   17                  18                  19                  20                  21                  22                  23 24 25 26 27 28 29 30 next_up/down
0     2015-01-05  RTS-3.15  RTS-3.15                 1.0  0.9497652206697209  0.9468633252979013  0.8903106399022038  0.8867461070045356  0.8108169863702316  0.7687623530487039   0.7023272286899199   0.6582504084360612   0.5194621211255698                  0.0  0.07172479978064412  ...   0.3427549726376401  0.3859635092369386  0.5616424270813102   0.618663528659073  0.7104159764192439  0.7216579647888128  0.8760982074512448  1  0  0  0  0  0  0         down
1     2015-01-06  RTS-3.15  RTS-3.15                 1.0  0.9996134790400225

In [4]:
df_cc_query = df_f[['TRADEDATE', 'OPEN', 'LOW', 'HIGH', 'CLOSE']].copy()
importlib.reload(candle_code_Lihovidov)  # Перезагрузка модуля, чтобы не брал данные из кэша
df_cc = candle_code_Lihovidov.run(df_cc_query)
df_cc['NEXT_SIZE_BODY'] = df_cc.SIZE_BODY.shift(-1)  # Добавление колонки с размером тела следующей свечи

print(df_cc.to_string(max_rows=8, max_cols=30))

      TRADEDATE  SIZE_BODY      OPEN       LOW      HIGH     CLOSE  UP/DOWN CANDLE_CODE  F_0  F_1  F_2  F_3  F_4  F_5  F_6  NEXT_SIZE_BODY
0    2015-02-02     1460.0   71810.0   71110.0   75730.0   73270.0        1     1101110    1    1    0    1    1    1    0          4210.0
1    2015-02-03     4210.0   73280.0   72650.0   77770.0   77490.0        1     1111110    1    1    1    1    1    1    0          1720.0
2    2015-02-04     1720.0   77380.0   75230.0   79680.0   75660.0        0     0011001    0    0    1    1    0    0    1          4200.0
3    2015-02-05     4200.0   75450.0   73770.0   80110.0   79650.0        1     1111101    1    1    1    1    1    0    1          2550.0
...         ...        ...       ...       ...       ...       ...      ...         ...  ...  ...  ...  ...  ...  ...  ...             ...
2382 2024-08-01       60.0  110480.0  109950.0  111310.0  110420.0        0     0101010    0    1    0    1    0    1    0          1750.0
2383 2024-08-02     1750.0 

In [5]:
# df_cc_query = df_f[['TRADEDATE', 'OPEN', 'LOW', 'HIGH', 'CLOSE']].copy()
# importlib.reload(candle_code_Lihovidov)  # Перезагрузка модуля, чтобы не брал данные из кэша
# df_cc = candle_code_Lihovidov.run(df_cc_query)

df_cc['TRADEDATE'] = pd.to_datetime(df_cc['TRADEDATE'])
df_cc['TRADEDATE'] = df_cc['TRADEDATE'].dt.date  # Оставляем только дату
df_cc = df_cc[['TRADEDATE', 'NEXT_SIZE_BODY', 'F_0', 'F_1', 'F_2', 'F_3', 'F_4', 'F_5', 'F_6']].copy()

print(df_cc.to_string(max_rows=8, max_cols=30))

       TRADEDATE  NEXT_SIZE_BODY  F_0  F_1  F_2  F_3  F_4  F_5  F_6
0     2015-02-02          4210.0    1    1    0    1    1    1    0
1     2015-02-03          1720.0    1    1    1    1    1    1    0
2     2015-02-04          4200.0    0    0    1    1    0    0    1
3     2015-02-05          2550.0    1    1    1    1    1    0    1
...          ...             ...  ...  ...  ...  ...  ...  ...  ...
2382  2024-08-01          1750.0    0    1    0    1    0    1    0
2383  2024-08-02          2470.0    0    0    1    0    1    0    0
2384  2024-08-05             0.0    0    0    0    0    1    0    0
2385  2024-08-06             NaN    1    0    1    1    1    0    1


In [6]:
# Слияние DataFrame по полю TRADEDATE
df_rez = pd.merge(df_rez, df_cc, on='TRADEDATE', how='inner')

# print(df_rez.to_string(max_rows=14, max_cols=30))
# print(df_rez.shape)

In [7]:
# Фильтрация строк, где значения в столбцах SHORTNAME и next_name равны (исключение переходов контракта)
df_rez = df_rez[df_rez['SHORTNAME'] == df_rez['next_name']]

# print(df_rez.to_string(max_rows=14, max_cols=30))
# print(df_rez.shape)

In [8]:
def target(x_0, x_1, x_2):
    if x_0 == 0 and x_1 == 0 and x_2 == 0:
        return 0
    elif x_0 == 0 and x_1 == 0 and x_2 == 1:
        return 1
    elif x_0 == 0 and x_1 == 1 and x_2 == 0:
        return 2
    elif x_0 == 1 and x_1 == 0 and x_2 == 1:
        return 3
    elif x_0 == 1 and x_1 == 1 and x_2 == 0:
        return 4
    elif x_0 == 1 and x_1 == 1 and x_2 == 1:
        return 5
    
df_rez['target'] = df_rez.apply(lambda x: target(x['F_0'], x['F_1'], x['F_2']), axis=1).shift(-1)

# # Создания единого целевого столбца на основе бинарных признаков
# df_rez['combined_target'] = df_rez[['F_0', 'F_1', 'F_2']].apply(lambda row: int(''.join(map(str, row)), 2), axis=1).shift(-1)

# print(df_rez.to_string(max_rows=14, max_cols=30))
# print(df_rez.shape)

In [9]:
# df_rez = df_rez.drop(['TRADEDATE', 'SHORTNAME', 'next_name', 31], axis=1)
df_rez = df_rez.dropna()
df_rez = df_rez.reset_index(drop=True)

# # Определение нового списка имен колонок (цифры от 0 до количества колонок - 1)
# new_columns = [i for i in range(len(df_rez.columns))]
# # Переиндексация колонок цифрами
# df_rez.columns = new_columns

# Удаляем колонку и сохраняем её в переменную
col = df_rez.pop('next_up/down')
# Вставляем колонку на нужную позицию
df_rez.insert(3, 'next_up/down', col)

# Удаляем колонку и сохраняем её в переменную
col = df_rez.pop('NEXT_SIZE_BODY')
# Вставляем колонку на нужную позицию
df_rez.insert(4, 'next_size_body', col)

print(df_rez.to_string(max_rows=14, max_cols=40))
print(df_rez.shape)

       TRADEDATE SHORTNAME next_name next_up/down  next_size_body                   3                   4                   5                   6                   7                   8                   9                   10                   11                   12                    13                   14                   15                   16                  17  ...                  19                  20                  21                  22                  23 24 25 26 27 28 29 30 F_0  F_1  F_2  F_3  F_4  F_5  F_6  target
0     2015-02-02  RTS-3.15  RTS-3.15           up          4210.0                 1.0                 1.0  0.9585517884189677  0.9585129519592994  0.9095693036622782  0.9065788962678162  0.8250611674239776   0.7867004543865781   0.6635112043186143    0.645015340401569    0.4312303390422929                  0.0   0.1549477649617461  0.32709037244164824  0.3880150685463513  ...  0.5770126995223116    0.69635714008311  0.7212513107305139  0.7765447201833081

In [10]:
# Сохранение DF в файл без индекса
df_rez.to_csv(fr'../nn_features_and_target.csv', index=False, sep=';')